## **Vehicle Detection and Counting Using Background Subtraction**

**Check:** https://docs.opencv.org/4.x/d6/d17/group__cudabgsegm.html

In [1]:
import cv2
import numpy as np

In [2]:
def preprocess_frame(frame: np.ndarray, ksize: tuple = (5, 5)) -> np.ndarray:
    """
    Convert a frame to grayscale and apply Gaussian blur.
    """
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred_frame = cv2.GaussianBlur(gray_frame, ksize, 0)
    return blurred_frame


def get_center(x, y, w, h):
    """
    Calculate the center of a bounding rectangle.
    """
    cx = x + (w // 2)
    cy = y + (h // 2)
    return cx, cy

In [3]:
# Parameters
x_start = 25
x_end = 1160
line_position = 550  # Position of the counting line

detection_points = []  # Store detected vehicle centers
vehicle_count = 0
offset = 5  # Tolerance around the line for counting
video_path = "../Data/Videos/Cars_Moving_1.mp4"

In [4]:
cap = cv2.VideoCapture(video_path)

# Background subtractor
background_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    processed_frame = preprocess_frame(frame)

    # Apply background subtraction and thresholding
    bg_sub = background_subtractor.apply(processed_frame)
    _, bg_sub = cv2.threshold(bg_sub, 25, 255, cv2.THRESH_BINARY)

    dilated_frame = cv2.dilate(bg_sub, (5, 5), iterations=1)

    contours, _ = cv2.findContours(
        dilated_frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
    )

    # Draw the counting line
    cv2.line(frame, (x_start, line_position), (x_end, line_position), (176, 130, 39), 2)

    # Process each contour
    for contour in contours:
        if cv2.contourArea(contour) < 2200:
            continue

        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        cx, cy = get_center(x, y, w, h)
        cv2.circle(frame, (cx, cy), 3, (0, 255, 0), -1)
        detection_points.append((cx, cy))

        # Check if the center crosses the counting line
        for cx, cy in detection_points:
            if line_position - offset < cy < line_position + offset:
                vehicle_count += 1
                cv2.line(
                    frame,
                    (x_start, line_position),
                    (x_end, line_position),
                    (0, 127, 255),
                    3,
                )
                detection_points.remove((cx, cy))
                # print("Number of Vehicles Detected:", vehicle_count)

    text = f"VEHICLE COUNT: {vehicle_count}"
    cv2.putText(frame, text, (320, 70), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 255), 4)

    cv2.imshow("Background Mask", bg_sub)
    cv2.imshow("Dilated Frame", dilated_frame)
    cv2.imshow("Original Video", frame)

    if cv2.waitKey(1) in [ord("q"), 27]:
        # print("Number of Vehicles Detected:", vehicle_count)
        break


cap.release()
cv2.destroyAllWindows()